# Weather AUS

Importa as bibliotecas necessárias:

 - pandas e numpy: Para manipulação de dados e operações numéricas.

 - tensorflow e keras: Para construir, compilar e treinar o modelo de RNN.

 - sklearn.preprocessing e sklearn.model_selection: Para normalização e divisão dos dados.

 - matplotlib.pyplot: Para visualização dos resultados.

In [1]:
import os
import random
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.optimizers import Adam, RMSprop
from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM, Dense
import matplotlib.pyplot as plt


# Definir a seed para reprodutibilidade
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

initializer = tf.keras.initializers.GlorotUniform(seed=seed)

tf.config.experimental.enable_op_determinism()

2025-03-08 20:29:36.482855: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-08 20:29:36.515167: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-08 20:29:36.555706: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741465776.700211   53376 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741465776.717545   53376 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-08 20:29:36.805177: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Carrega o conjunto de dados weatherAUS.csv.

Filtra os dados para incluir apenas informações da localidade Albury.

Seleciona as colunas Humidity9am e Humidity3pm como variáveis de entrada e saída.

In [2]:
# Carregar dados do arquivo CSV
weather_data = pd.read_csv('/workspaces/usp-dsa-ann/data/weatherAUS.csv')

In [3]:
weather_data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [4]:
# Extrair colunas de interesse e linhas da localidade Albury
data = weather_data.loc[weather_data['Location'] == 'Albury']
data = data.iloc[:,13:15]
data.head()

,Humidity9am,Humidity3pm
0,71.0,22.0
1,44.0,25.0
2,38.0,30.0
3,45.0,16.0
4,82.0,33.0


# Pré-Processamento

 - Remove valores ausentes (NaN) do conjunto de dados.

 - Converte as variáveis de entrada (Humidity9am) e saída (Humidity3pm) em arrays NumPy.

 - Normaliza os dados para o intervalo [0, 1] usando MinMaxScaler.

In [5]:
# Pre-processamento dos dados
data_cleaned = data.dropna()
x = data_cleaned['Humidity9am'].values.reshape(-1, 1)
y = data_cleaned['Humidity3pm'].values.reshape(-1, 1)

# Normalização dos dados
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
x_scaled = scaler_x.fit_transform(x)
y_scaled = scaler_y.fit_transform(y)

# Divisão em conjunto de treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.2, random_state=42)

# Ajuste do shape para RNN (supondo uma janela de tempo de 4)
time_steps = 4
samples_train = x_train.shape[0] // time_steps  # Número de amostras ajustadas para múltiplos de time_steps

# Redimensionando os dados para RNN
x_train_rnn = x_train[:samples_train * time_steps].reshape(samples_train, time_steps, 1)
y_train_rnn = y_train[:samples_train * time_steps].reshape(samples_train, time_steps, 1)

# Test set - mantendo sequência temporal unitária
x_test_rnn = x_test.reshape(x_test.shape[0], 1, 1)

 - Divide os dados em conjuntos de treinamento e teste (80% e 20%).

 - Redimensiona os dados de entrada para o formato esperado pela RNN ([amostras, timesteps, features]).

Modelo da RNN

Define um modelo sequencial composto por:

 - Uma camada SimpleRNN com 20 neurônios e ativação ReLU.

 - Três camadas densas para refinar as previsões.

 - Uma camada de saída com um único neurônio (regressão).

In [6]:
# Criar o modelo LSTM
model = Sequential([
    SimpleRNN(units=20, activation = 'relu', return_sequences=True, input_shape=(4, 1)),
    Dense(units=10, activation = 'relu'),
    Dense(units=5, activation = 'relu'),
    Dense(1)
])

2025-03-08 20:35:11.797040: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Compila e treina

 - Define o otimizador Adam com uma taxa de aprendizado personalizada.
 - Compila o modelo com a função de perda mean_squared_error.
 - Treina o modelo por 20 épocas com tamanho de batch de 32.

In [7]:
learning_rate = 0.00001
optimizer = Adam(learning_rate=learning_rate)

# Compilar modelo
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Treinar modelo
history = model.fit(x_train_rnn, y_train_rnn, epochs=20, batch_size=32, verbose=0)

2025-03-08 20:35:53.865602: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


# Previsão

In [8]:
# Predição
Y_predicted = model.predict(x_test)

# Desescalonar os resultados
Y_test_descaled = scaler_y.inverse_transform(y_test)

# Reshape
Y_predicted = Y_predicted.reshape(Y_predicted.shape[0],Y_predicted.shape[1])
Y_predicted_descaled = scaler_y.inverse_transform(Y_predicted)

 1/19 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 

2025-03-08 20:36:09.307285: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


 - Define uma função para calcular o coeficiente de determinação (R²).

 - Avalia a performance do modelo ao comparar os valores previstos com os reais.

In [9]:
# Calcular R-squared
def r_squared(y_actual, y_predict):
    corr = np.corrcoef(y_actual.flatten(), y_predict.flatten())[0, 1]
    return corr ** 2

r2_score = r_squared(Y_test_descaled, Y_predicted_descaled)
print(f'R-squared: {r2_score}')

R-squared: 0.5726577729142217
